In [ ]:
"""
================================================================================
PROYEK      : Estimasi Debit Sungai Metode CMHC Berbasis Citra Sentinel-2
MODUL       : Langkah D - Pemodelan, Evaluasi Akurasi, & Visualisasi
DESKRIPSI   : Script ini menjalankan pipeline utama metode CMHC:
              1. Menggabungkan data debit bersih (Langkah B) & fitur spektral (Langkah C).
              2. Melatih model Random Forest untuk klasifikasi kelas debit (Rendah/Menengah/Tinggi).
              3. Membangun model regresi spesifik per kelas (Cubic/Power/Exponential).
              4. Mengevaluasi akurasi (NSE, RMSE, RRMSE) pada data uji (2022-2024).
              5. Menghasilkan Hidrograf, Scatter Plot, dan Confusion Matrix.
================================================================================

DEPENDENSI:
  - pandas, numpy, matplotlib, seaborn
  - scikit-learn (RandomForest, Metrics)
  - scipy (stats)
"""

# ==============================================================================
# 1. IMPOR LIBRARY
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import os
import json
import warnings
import traceback
import joblib
from pathlib import Path
from google.colab import drive

# Import dari Scikit-Learn
from sklearn.model_selection import GroupKFold, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy.stats import randint
from imblearn.pipeline import Pipeline  # Membutuhkan imbalanced-learn

# Konfigurasi Tampilan
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# ==============================================================================
# 2. KONFIGURASI GLOBAL
# ==============================================================================

# Folder Kerja (Sesuaikan dengan path Google Drive Anda)
GEE_EXPORT_FOLDER = '/content/drive/MyDrive/GEE_Exports_Tesis_Master/'
OUTPUT_REPORTS_DIR = os.path.join(GEE_EXPORT_FOLDER, 'Laporan_Skenario_LangkahD')

# Pilih Stasiun yang Ingin Diproses (Bisa satu atau semua)
ROI_NAME = 'Kalibawang'  # Opsi: 'Kalibawang', 'Bendungan', 'Kedungmiri'

# Mapping Nama untuk Judul Grafik
RIVER_MAP = {
    'Kalibawang': 'Sungai Progo (Stasiun Kalibawang)',
    'Kedungmiri': 'Sungai Oyo (Stasiun Kedungmiri)',
    'Bendungan' : 'Sungai Serang (Stasiun Bendungan)'
}

# Periode Data (Harus sama dengan Langkah B & C)
TRAIN_TAG = '2019_2021'
TEST_TAG  = '2022_2024'

# Skenario Analisis
SCENARIOS = ['Jenks_Gabungan', 'Jenks_Musiman']

# Parameter Training
MIN_SAMPLES_FOR_RF  = 5   # Batas minimum sampel agar training jalan (PENTING untuk Oyo Kemarau)
WARN_THRESHOLD      = 19  # Batas untuk memunculkan peringatan data sedikit
RNG                 = 42  # Random State untuk reprodusibilitas

# Kolom Data
DEBIT_COL   = 'Debit_Aktual'
TARGET_COL  = 'Kelas_Aktual'
CLASS_LABELS = ['Rendah', 'Menengah', 'Tinggi']

# Fitur Spektral (Input RF)
FEATURES = [
    'Nilai C', 'Nilai W', 'Nilai Mlow', 'Nilai Mmedium', 'Nilai Mhigh',
    'MWlow', 'MWmedium', 'MWhigh'
]
ALL_COLS = FEATURES + ['Rasio C/Mlow', 'Rasio C/Mmedium', 'Rasio C/Mhigh']

# ==============================================================================
# 3. FUNGSI BANTUAN (UTILITIES)
# ==============================================================================

def mount_drive():
    try:
        drive.mount('/content/drive', force_remount=True)
    except: pass

def check_files(file_list):
    """Memastikan semua file input tersedia."""
    missing = [f for f in file_list if not os.path.exists(f)]
    if missing:
        print("❌ FILE HILANG:")
        for m in missing: print(f"   - {os.path.basename(m)}")
        return False
    return True

def standardize_date(df):
    """Format tanggal menjadi datetime standar."""
    col = next((c for c in df.columns if c.lower() in ['date', 'tanggal']), None)
    if col:
        df['Tanggal'] = pd.to_datetime(df[col], errors='coerce')
        if col != 'Tanggal': df = df.drop(columns=[col])
        return df.dropna(subset=['Tanggal'])
    return df

def ensure_season_col(df):
    """Memastikan kolom Musim ada (Hujan/Kemarau)."""
    if 'Musim' in df.columns:
        # Standarisasi label
        df['Musim'] = df['Musim'].astype(str).str.strip().str.capitalize()
        df['Musim'] = df['Musim'].replace({'Wet': 'Hujan', 'Dry': 'Kemarau'})
        # Filter hanya Hujan/Kemarau
        return df[df['Musim'].isin(['Hujan', 'Kemarau'])]

    # Fallback jika kolom tidak ada (berdasarkan bulan)
    df['Musim'] = np.where(df['Tanggal'].dt.month.isin([11,12,1,2,3,4]), 'Hujan', 'Kemarau')
    return df

def add_actual_class(df, thresholds, is_seasonal):
    """Menambahkan kolom Kelas Aktual berdasarkan threshold Jenks."""
    df = df.copy()
    df[TARGET_COL] = pd.NA

    def get_cls(val, q1, q2):
        if pd.isna(val): return pd.NA
        return 'Tinggi' if val >= q2 else 'Menengah' if val >= q1 else 'Rendah'

    if is_seasonal:
        for seas in ['Hujan', 'Kemarau']:
            if seas in thresholds:
                q1, q2 = thresholds[seas]['q1'], thresholds[seas]['q2']
                mask = df['Musim'] == seas
                df.loc[mask, TARGET_COL] = df.loc[mask, DEBIT_COL].apply(lambda x: get_cls(x, q1, q2))
    else:
        # Skenario Gabungan
        q1, q2 = thresholds['TANPA_MUSIM']['q1'], thresholds['TANPA_MUSIM']['q2']
        df[TARGET_COL] = df[DEBIT_COL].apply(lambda x: get_cls(x, q1, q2))

    return df.dropna(subset=[TARGET_COL])

# ==============================================================================
# 4. FUNGSI PEMODELAN (TRAINING & PREDICTION)
# ==============================================================================

def train_rf_classifier(df, name):
    """Melatih Random Forest Classifier."""
    print(f"\n--- Training RF: {name} ---")
    df = df.dropna(subset=FEATURES + [TARGET_COL])

    n_samples = len(df)
    print(f"   Jumlah Sampel: {n_samples}")
    print(f"   Distribusi Kelas:\n{df[TARGET_COL].value_counts()}")

    if n_samples < MIN_SAMPLES_FOR_RF:
        print(f"   ❌ GAGAL: Sampel < {MIN_SAMPLES_FOR_RF}. Model tidak dilatih.")
        return None

    if n_samples < WARN_THRESHOLD:
        print(f"   ⚠️ WARNING: Sampel sedikit (<{WARN_THRESHOLD}).")

    # Pipeline
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('rf', RandomForestClassifier(random_state=RNG, n_jobs=-1, class_weight='balanced'))
    ])

    # Cross-Validation Strategy
    # Jika data sangat sedikit (<10), kurangi fold CV jadi 2 atau skip CV
    cv_splits = 3 if n_samples >= 10 else 2
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=RNG)

    param_grid = {
        'rf__n_estimators': [100, 200, 300],
        'rf__max_depth': [None, 5, 10],
        'rf__min_samples_split': [2, 5]
    }

    try:
        search = RandomizedSearchCV(pipeline, param_grid, n_iter=10, cv=cv, scoring='balanced_accuracy', random_state=RNG, n_jobs=-1)
        search.fit(df[FEATURES], df[TARGET_COL])
        print(f"   ✅ Sukses. Best CV Score: {search.best_score_:.3f}")
        return search.best_estimator_
    except Exception as e:
        print(f"   ⚠️ CV Gagal ({e}). Fallback ke training langsung.")
        pipeline.fit(df[FEATURES], df[TARGET_COL])
        return pipeline

def train_regressions(df, season_name):
    """Melatih 3 model regresi (Rendah, Menengah, Tinggi)."""
    river_full = RIVER_MAP.get(ROI_NAME, ROI_NAME)
    print(f"\n--- Training Regresi: {river_full} [{season_name}] ---")

    params = {}

    # 1. KELAS RENDAH (Cubic Polynomial)
    # Rumus: y = ax^3 + bx^2 + cx + d
    d_low = df[df[TARGET_COL] == 'Rendah'].dropna(subset=['Rasio C/Mlow', DEBIT_COL])
    if len(d_low) >= 3:
        try:
            coeffs = np.polyfit(d_low['Rasio C/Mlow'], d_low[DEBIT_COL], 3)
            params['Rendah'] = coeffs.tolist()
        except: params['Rendah'] = None
    else: params['Rendah'] = None

    # 2. KELAS MENENGAH (Power Law)
    # Rumus: y = a * x^b -> log(y) = log(a) + b*log(x)
    d_med = df[(df[TARGET_COL] == 'Menengah') & (df['Rasio C/Mmedium'] > 0)].dropna(subset=['Rasio C/Mmedium', DEBIT_COL])
    if len(d_med) >= 3:
        try:
            # Polyfit pada log-log space
            # coeffs_log[0] = slope (b), coeffs_log[1] = intercept (log a)
            coeffs_log = np.polyfit(np.log(d_med['Rasio C/Mmedium']), np.log(d_med[DEBIT_COL]), 1)
            a = np.exp(coeffs_log[1])
            b = coeffs_log[0]
            params['Menengah'] = [a, b] # Simpan [a, b]
        except: params['Menengah'] = None
    else: params['Menengah'] = None

    # 3. KELAS TINGGI (Exponential)
    # Rumus: y = a * exp(b * x) -> log(y) = log(a) + b * x
    d_high = df[(df[TARGET_COL] == 'Tinggi') & (df[DEBIT_COL] > 0)].dropna(subset=['Rasio C/Mhigh', DEBIT_COL])
    if len(d_high) >= 3:
        try:
            # Polyfit pada semi-log space (x vs log y)
            # coeffs_log[0] = slope (b), coeffs_log[1] = intercept (log a)
            coeffs_log = np.polyfit(d_high['Rasio C/Mhigh'], np.log(d_high[DEBIT_COL]), 1)
            a = np.exp(coeffs_log[1])
            b = coeffs_log[0]
            params['Tinggi'] = [a, b] # Simpan [a, b]
        except: params['Tinggi'] = None
    else: params['Tinggi'] = None

    print(f"   Model Terbentuk: {[k for k, v in params.items() if v is not None]}")
    return params

def predict_discharge(row, models_rf, params_reg, scenario):
    """Fungsi pembungkus untuk memprediksi kelas lalu nilai debit."""
    # 1. Prediksi Kelas
    musim = row.get('Musim', 'TANPA_MUSIM')
    features = row[FEATURES].values.reshape(1, -1)

    rf_model = None
    if scenario == 'Jenks_Gabungan':
        rf_model = models_rf.get('main')
    else:
        rf_model = models_rf.get(musim.lower()) # 'hujan' atau 'kemarau'

    if not rf_model: return pd.NA, np.nan

    try:
        pred_class = rf_model.predict(features)[0]
    except: return pd.NA, np.nan

    # 2. Estimasi Nilai Debit (Regresi)
    reg_p = params_reg['TANPA_MUSIM'] if scenario == 'Jenks_Gabungan' else params_reg.get(musim)
    if not reg_p or not reg_p.get(pred_class): return pred_class, np.nan

    coef = reg_p[pred_class]
    val = np.nan

    try:
        if pred_class == 'Rendah':
            x = row['Rasio C/Mlow']
            val = coef[0]*x**3 + coef[1]*x**2 + coef[2]*x + coef[3]
        elif pred_class == 'Menengah':
            x = row['Rasio C/Mmedium']
            val = coef[0] * (x ** coef[1])
        elif pred_class == 'Tinggi':
            x = row['Rasio C/Mhigh']
            val = coef[0] * np.exp(coef[1] * x)
    except: pass

    return pred_class, max(0, val) # Debit tidak boleh negatif

# ==============================================================================
# 5. FUNGSI VISUALISASI (BAHASA INDONESIA)
# ==============================================================================

def calc_metrics(obs, sim):
    """Menghitung NSE, RMSE, RRMSE."""
    valid = np.isfinite(obs) & np.isfinite(sim)
    o, s = obs[valid], sim[valid]
    if len(o) == 0: return np.nan, np.nan, np.nan, 0

    rmse = np.sqrt(np.mean((o - s)**2))
    rrmse = (rmse / np.mean(o)) * 100 if np.mean(o) != 0 else np.nan
    nse = 1 - (np.sum((o - s)**2) / np.sum((o - np.mean(o))**2))

    return nse, rmse, rrmse, len(o)

def plot_hydrograph(df, title_suffix, output_dir):
    """Membuat Hidrograf Aliran."""
    df = df.sort_values('Tanggal').dropna(subset=['Debit_Stasiun', 'Debit_Estimasi'])
    if df.empty: return

    nse, rmse, rrmse, n = calc_metrics(df['Debit_Stasiun'].values, df['Debit_Estimasi'].values)

    plt.figure(figsize=(10, 5))
    plt.plot(df['Tanggal'], df['Debit_Stasiun'], 'k-', label='Debit Observasi', alpha=0.7)
    plt.plot(df['Tanggal'], df['Debit_Estimasi'], 'r--', label='Debit Estimasi', marker='.')

    title = f"Hidrograf Aliran - {RIVER_MAP.get(ROI_NAME, ROI_NAME)}\n({title_suffix})"
    plt.title(title, fontsize=12, fontweight='bold')
    plt.ylabel("Debit ($m^3/s$)")
    plt.xlabel("Tanggal")

    # Statistik Box
    stats = f"NSE = {nse:.2f}\nRMSE = {rmse:.2f} $m^3/s$\nRRMSE = {rrmse:.2f}%\nn = {n}"
    plt.annotate(stats, xy=(0.02, 0.95), xycoords='axes fraction',
                 bbox=dict(boxstyle="round", fc="white", alpha=0.8), va='top')

    plt.legend()
    plt.grid(True, linestyle=':')

    fname = f"Hidrograf_{ROI_NAME}_{title_suffix.replace(' ', '_')}.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   📊 Grafik tersimpan: {fname}")

def plot_scatter(df, title_suffix, output_dir):
    """Membuat Scatter Plot 1:1."""
    df = df.dropna(subset=['Debit_Stasiun', 'Debit_Estimasi'])
    if df.empty: return

    nse, rmse, rrmse, n = calc_metrics(df['Debit_Stasiun'].values, df['Debit_Estimasi'].values)

    plt.figure(figsize=(6, 6))
    plt.scatter(df['Debit_Stasiun'], df['Debit_Estimasi'], alpha=0.6, edgecolors='w')

    # Garis 1:1
    mx = max(df['Debit_Stasiun'].max(), df['Debit_Estimasi'].max()) * 1.1
    plt.plot([0, mx], [0, mx], 'k--', label='Garis 1:1')

    title = f"Scatter Plot - {RIVER_MAP.get(ROI_NAME, ROI_NAME)}\n({title_suffix})"
    plt.title(title, fontsize=12, fontweight='bold')
    plt.xlabel("Debit Observasi ($m^3/s$)")
    plt.ylabel("Debit Estimasi ($m^3/s$)")
    plt.xlim(0, mx); plt.ylim(0, mx)

    stats = f"NSE = {nse:.2f}\nRMSE = {rmse:.2f}\nRRMSE = {rrmse:.2f}%\nn = {n}"
    plt.annotate(stats, xy=(0.05, 0.95), xycoords='axes fraction',
                 bbox=dict(boxstyle="round", fc="white", alpha=0.8), va='top')

    plt.grid(True, linestyle=':')
    fname = f"Scatter_{ROI_NAME}_{title_suffix.replace(' ', '_')}.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()

# ==============================================================================
# 6. EKSEKUSI PIPELINE UTAMA
# ==============================================================================

def main():
    mount_drive()
    Path(OUTPUT_REPORTS_DIR).mkdir(parents=True, exist_ok=True)

    print("="*60)
    print(f"🚀 MULAI PEMODELAN CMHC: {ROI_NAME.upper()}")
    print("="*60)

    # 1. Load File Thresholds
    roi_lower = ROI_NAME.lower()
    thresh_file = os.path.join(GEE_EXPORT_FOLDER, f'kandidat_thresholds_{roi_lower}_FIX.json')

    if not os.path.exists(thresh_file):
        print("❌ File Threshold JSON tidak ditemukan. Jalankan Langkah B dulu.")
        return

    with open(thresh_file, 'r') as f: thresholds_db = json.load(f)

    # 2. Loop Skenario
    for scenario in SCENARIOS:
        print(f"\n🔄 Memproses Skenario: {scenario}...")

        is_seasonal = 'Musiman' in scenario
        thresholds = thresholds_db.get(scenario, {})

        if not thresholds:
            print(f"   ⚠️ Threshold untuk {scenario} kosong di JSON.")
            continue

        # Output Sub-folder
        scenario_dir = os.path.join(OUTPUT_REPORTS_DIR, f"{scenario}_TERBAIK")
        Path(scenario_dir).mkdir(exist_ok=True)

        # 3. Load Data Latih & Uji
        # Note: Load fitur dari Langkah C (sesuai skenario)
        #       Load debit dari Langkah B (file debit bersih)

        # --- LOGIKA LOADING DATA YANG DINAMIS ---
        # Kita butuh membaca file 'tabel_fitur_...' dari Langkah C.
        # Skenario Gabungan -> file 'TANPA_MUSIM'
        # Skenario Musiman  -> file 'Hujan' dan 'Kemarau'

        features_latih_list = []
        features_uji_list = []

        seasons_to_load = ['Hujan', 'Kemarau'] if is_seasonal else ['TANPA_MUSIM']

        for s in seasons_to_load:
            f_latih = os.path.join(GEE_EXPORT_FOLDER, f"tabel_fitur_latih_{roi_lower}_{s}_{TRAIN_TAG}_individu.csv")
            f_uji   = os.path.join(GEE_EXPORT_FOLDER, f"tabel_fitur_uji_{roi_lower}_{s}_{TEST_TAG}_individu.csv")

            if os.path.exists(f_latih):
                df = pd.read_csv(f_latih)
                if 'Musim' not in df.columns: df['Musim'] = s if is_seasonal else 'TANPA_MUSIM'
                features_latih_list.append(df)

            if os.path.exists(f_uji):
                df = pd.read_csv(f_uji)
                if 'Musim' not in df.columns: df['Musim'] = s if is_seasonal else 'TANPA_MUSIM'
                features_uji_list.append(df)

        if not features_latih_list:
            print("   ❌ File fitur latih tidak ditemukan.")
            continue

        df_feat_latih = pd.concat(features_latih_list)
        df_feat_uji   = pd.concat(features_uji_list)

        # Load Debit Bersih
        f_debit_latih = os.path.join(GEE_EXPORT_FOLDER, f"data_debit_latih_{roi_lower}_{TRAIN_TAG}_FIX.csv")
        f_debit_uji   = os.path.join(GEE_EXPORT_FOLDER, f"data_debit_uji_{roi_lower}_{TEST_TAG}_FIX.csv")

        df_debit_latih_clean = pd.read_csv(f_debit_latih)
        df_debit_uji_clean   = pd.read_csv(f_debit_uji)

        # Merge Fitur + Debit
        df_feat_latih = standardize_date(df_feat_latih)
        df_debit_latih_clean = standardize_date(df_debit_latih_clean)
        df_latih = pd.merge(df_feat_latih, df_debit_latih_clean, on='Tanggal')

        df_feat_uji = standardize_date(df_feat_uji)
        df_debit_uji_clean = standardize_date(df_debit_uji_clean)
        df_uji = pd.merge(df_feat_uji, df_debit_uji_clean, on='Tanggal')

        # Pastikan kolom Musim & Kelas Aktual
        df_latih = ensure_season_col(df_latih)
        df_uji   = ensure_season_col(df_uji)

        df_latih = add_actual_class(df_latih, thresholds, is_seasonal)
        # Data uji tidak butuh kelas aktual untuk prediksi, tapi butuh untuk evaluasi confusion matrix
        df_uji   = add_actual_class(df_uji, thresholds, is_seasonal)

        # 4. Training Model (RF & Regresi)
        models_rf = {}
        reg_params = {}

        if is_seasonal:
            # --- MUSIM HUJAN ---
            df_h = df_latih[df_latih['Musim'] == 'Hujan']
            if not df_h.empty:
                models_rf['hujan'] = train_rf_classifier(df_h, "RF Musim Hujan")
                q1, q2 = thresholds['Hujan']['q1'], thresholds['Hujan']['q2']
                reg_params['Hujan'] = train_regressions(df_h, "Musim Hujan")

            # --- MUSIM KEMARAU ---
            df_k = df_latih[df_latih['Musim'] == 'Kemarau']
            if not df_k.empty:
                models_rf['kemarau'] = train_rf_classifier(df_k, "RF Musim Kemarau")
                q1, q2 = thresholds['Kemarau']['q1'], thresholds['Kemarau']['q2']
                reg_params['Kemarau'] = train_regressions(df_k, "Musim Kemarau")
        else:
            # --- GABUNGAN ---
            models_rf['main'] = train_rf_classifier(df_latih, "RF Tanpa Musim")
            q1, q2 = thresholds['TANPA_MUSIM']['q1'], thresholds['TANPA_MUSIM']['q2']
            reg_params['TANPA_MUSIM'] = train_regressions(df_latih, "Tanpa Musim")

        # 5. Prediksi pada Data Uji
        print("\n--- Prediksi Data Uji ---")
        results = []
        for _, row in df_uji.iterrows():
            pred_cls, pred_val = predict_discharge(row, models_rf, reg_params, scenario)
            res = row.to_dict()
            res['Kelas_Prediksi'] = pred_cls
            res['Debit_Estimasi'] = pred_val
            res['Debit_Stasiun']  = row[DEBIT_COL] # Rename biar konsisten
            results.append(res)

        df_eval = pd.DataFrame(results)

        # 6. Evaluasi & Visualisasi
        if is_seasonal:
            # Plot per musim
            for s in ['Hujan', 'Kemarau']:
                sub_df = df_eval[df_eval['Musim'] == s]
                if not sub_df.empty:
                    plot_hydrograph(sub_df, f"Musim {s}", scenario_dir)
                    plot_scatter(sub_df, f"Musim {s}", scenario_dir)
        else:
            # Plot Gabungan
            plot_hydrograph(df_eval, "Tanpa Musim", scenario_dir)
            plot_scatter(df_eval, "Tanpa Musim", scenario_dir)

        # Simpan CSV Hasil Evaluasi
        out_csv = os.path.join(scenario_dir, f"Evaluasi_{ROI_NAME}_{scenario}.csv")
        df_eval.to_csv(out_csv, index=False)
        print(f"✅ Selesai {scenario}. Hasil di: {scenario_dir}")

    print("\n🏁 SEMUA PROSES SELESAI.")

if __name__ == "__main__":
    main()